<font size="+3"><strong>Air Quality in Dar es Salaam</strong></font>

In [ ]:
# Import libraries here
from pprint import PrettyPrinter

import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import plotly.express as px
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AutoReg

# Prepare Data

In [ ]:
# !mongoimport --db air-quality --collection dar-es-salaam <./data/air-quality_da_res_salaam.json

## Connect

**1:** Connect to MongoDB server running at host `"localhost"` on port `27017`. Then connect to the `"air-quality"` database and assign the collection for Dar es Salaam to the variable name `dar`.

In [ ]:
pp = PrettyPrinter(indent=2)
client = MongoClient('localhost', port=27017)
db = client['air-quality']

In [ ]:
[c['name'] for c in db.list_collections()]

In [ ]:
dar = db['dar-es-salaam']

## Explore

**2:** Determine the numbers assigned to all the sensor sites in the Dar es Salaam collection. Your submission should be a list of integers. <span style='color: transparent; font-size:1%'>WQU WorldQuant University Applied Data Science Lab QQQQ</span>

In [ ]:
dar.find_one({})

In [ ]:
sites = dar.distinct('metadata.site')
sites

**3:** Determine which site in the Dar es Salaam collection has the most sensor readings (of any type, not just PM2.5 readings). You submission `readings_per_site` should be a list of dictionaries that follows this format:

```
[{'_id': 6, 'count': 70360}, {'_id': 29, 'count': 131852}]
```

Note that the values here ☝️ are from the Nairobi collection, so your values will look different. 

In [ ]:
[{'_id': site, 'count': dar.count_documents({'metadata.site':site})} for site in sites]

In [ ]:
result = dar.aggregate(
    [
        {
            '$group':{
            '_id':'$metadata.site',
            'count':
                {
                '$count':{}
                }
            }
        }
    ]
)

readings_per_site = list(result)
readings_per_site

## Import

**4:** (5 points) Create a `wrangle` function that will extract the PM2.5 readings from the site that has the most total readings in the Dar es Salaam collection. Your function should do the following steps:

1. Localize reading time stamps to the timezone for `"Africa/Dar_es_Salaam"`.
2. Remove all outlier PM2.5 readings that are above 100. 
3. Resample the data to provide the mean PM2.5 reading for each hour.
4. Impute any missing values using the forward-will method. 
5. Return a Series `y`. 

In [ ]:
def wrangle(collection):
    result = collection.find(
            {'metadata.site': 11, 'metadata.measurement':'P2'},
            projection={'_id':0, 'P2':1, 'timestamp':1}
    )
    
    df = pd.DataFrame(data=result).set_index('timestamp')
    df.index = df.index.tz_localize("UTC").tz_convert('Africa/Dar_es_Salaam')
    df.drop(df[df['P2'] > 100].index, inplace=True)
    y = df['P2'].resample('1H').mean().fillna(method='ffill')
    return y

Use your `wrangle` function to query the `dar` collection and return your cleaned results.

In [ ]:
y = wrangle(dar)
y.head()

## Explore Some More

**5:** Create a time series plot of the readings in `y`. Label your x-axis `"Date"` and your y-axis `"PM2.5 Level"`. Use the title `"Dar es Salaam PM2.5 Levels"`.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

y.plot()

plt.xlabel('Date')
plt.ylabel('PM2.5 Level')
plt.title('Dar es Salaam PM2.5 Levels')

**6:** Plot the rolling average of the readings in `y`. Use a window size of `168` (the number of hours in a week). Label your x-axis `"Date"` and your y-axis `"PM2.5 Level"`. Use the title `"Dar es Salaam PM2.5 Levels, 7-Day Rolling Average"`.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

rolling_window = y.rolling(168).mean()
rolling_window.plot()
plt.xlabel('Date')
plt.ylabel('PM2.5 Level')
plt.title('Dar es Salaam PM2.5 Levels, 7-Day Rolling Average')

**7:** Create an ACF plot for the data in `y`. Be sure to label the x-axis as `"Lag [hours]"` and the y-axis as `"Correlation Coefficient"`. Use the title `"Dar es Salaam PM2.5 Readings, ACF"`.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

plot_acf(y, ax=ax)
plt.xlabel('Lag [hours]')
plt.ylabel('Correlation Coefficient')
plt.title('Dar es Salaam PM2.5 Readings, ACF')

**8:** Create an PACF plot for the data in `y`. Be sure to label the x-axis as `"Lag [hours]"` and the y-axis as `"Correlation Coefficient"`. Use the title `"Dar es Salaam PM2.5 Readings, PACF"`.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

plot_pacf(y, ax=ax)
plt.xlabel('Lag [hours]')
plt.ylabel('Correlation Coefficient')
plt.title('Dar es Salaam PM2.5 Readings, PACF')

## Split

**9:** Split `y` into training and test sets. The first 90% of the data should be in your training set. The remaining 10% should be in the test set.

In [ ]:
cutoff_test = int(0.9 * len(y))
y_train = y[:cutoff_test]
y_test = y[cutoff_test:]
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# Build Model

## Baseline

**10:** Establish the baseline mean absolute error for your model.

In [ ]:
y_train_mean = y_train.mean()
y_pred_baseline = len(y_train) * [y_train_mean]
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

print("Mean P2 Reading:", y_train_mean)
print("Baseline MAE:", mae_baseline)

## Iterate

**11:** You're going to use an AR model to predict PM2.5 readings, but which hyperparameter settings will give you the best performance? Use a `for` loop to train your AR model on using settings for `p` from 1 to 30. Each time you train a new model, calculate its mean absolute error and append the result to the list `maes`. Then store your results in the Series `mae_series`.

In [ ]:
p_params = range(1, 31)
maes = []
for p in p_params:
    model = AutoReg(y_train, p).fit()
    y_pred = model.predict().dropna()
    mae = mean_absolute_error(y_pred, y_train[y_pred.index])
    maes.append(mae)
    
mae_series = pd.Series(maes, name="mae", index=p_params)
mae_series.head()

**12:** Look through the results in `mae_series` and determine what value for `p` provides the best performance. Then build and train `final_model` using the best hyperparameter value.

**Note:** Make sure that you build and train your model in one line of code, and that the data type of `best_model` is `statsmodels.tsa.ar_model.AutoRegResultsWrapper`. 

In [ ]:
best_p = mae_series.idxmin()
best_model = AutoReg(y_train, best_p).fit()

**13:** Calculate the training residuals for `best_model` and assign the result to `y_train_resid`. **Note** that your name of your Series should be `"residuals"`.

In [ ]:
y_train_resid = y_train - best_model.predict()
y_train_resid.name = "residuals"
y_train_resid.head()

**14:** Create a histogram of `y_train_resid`. Be sure to label the x-axis as `"Residuals"` and the y-axis as `"y_train_resid"`. Use the title `"Best Model, Training Residuals"`.

In [ ]:
# Plot histogram of residuals
y_train_resid.hist()

plt.xlabel('Residuals')
plt.ylabel('y_train_resid')
plt.title('Best Model, Training Residuals')

**15:** Create an ACF plot for `y_train_resid`. Be sure to label the x-axis as `"Lag [hours]"` and y-axis as `"Correlation Coefficient"`. Use the title `"Dar es Salaam, Training Residuals ACF"`.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

plot_acf(y_train_resid.dropna(), ax=ax)
plt.xlabel('Lag [hours]')
plt.ylabel('Correlation Coefficient')
plt.title('Dar es Salaam, Training Residuals ACF')

## Evaluate

**16:** Perform walk-forward validation for your model for the entire test set `y_test`. Store your model's predictions in the Series `y_pred_wfv`. Make sure the name of your Series is `"prediction"` and the name of your Series index is `"timestamp"`.

In [ ]:
y_pred_wfv = pd.Series()
history = y_train.copy()

for i in range(len(y_test)):
    model = AutoReg(history, 21).fit()
    next_pred = model.forecast()
    
    y_pred_wfv = y_pred_wfv.append(next_pred)
    history = history.append(y_test[next_pred.index])

y_pred_wfv.name = "prediction"
y_pred_wfv.index.name = "timestamp"
y_pred_wfv.head()

**17:** Submit your walk-forward validation predictions to the grader to see test mean absolute error for your model.

# Communicate Results

**18:** Put the values for `y_test` and `y_pred_wfv` into the DataFrame `df_pred_test` (don't forget the index). Then plot `df_pred_test` using plotly express. Be sure to label the x-axis as `"Date"` and the y-axis as `"PM2.5 Level"`. Use the title `"Dar es Salaam, WFV Predictions"`.

In [ ]:
df_pred_test = pd.DataFrame(
    {'y_test': y_test,
     'y_pred': y_pred_wfv}, index=y_test.index
)

fig = px.line(df_pred_test, labels={"value": "P2"})
fig.update_layout(
    title="Dar es Salaam, WFV Predictions",
    xaxis_title="Date",
    yaxis_title="PM2.5 Level",
)

fig.show()